# Предсказание является ли трафик атакой вида C&C

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score

In [14]:
from sklearn.tree import DecisionTreeClassifier

In [3]:
from sklearn.externals import joblib
# для сохранения обученных моделей

In [7]:
big = pd.read_csv('test', sep = '	')
big = big.drop('Unnamed: 0', axis = 1)

In [8]:
big

,day,hour,addr,port,addr.1,port.1,enum(icmp),enum(tcp),enum(upd),string.1(irc),...,set[string](-),set[string]((empty)),string.5(FileDownload),string.5(C&C-FileDownload),string.5(C&C-Torii),string.5(Attack),string.5(DDoS),string.5(PartOfAHorizontalPortScan),string.5(C&C),string.5(-)
0,298,12,6805,58687,369334,123,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,298,12,6805,1900,420753,1900,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,298,12,6805,32893,369335,123,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,298,12,6805,53395,316235,443,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,298,12,6805,52801,292458,53,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211508,10,21,6809,123,484566,123,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1211509,10,20,6809,51834,509175,23,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1211510,10,22,6809,123,611845,123,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1211511,10,22,6809,123,646453,123,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [163]:
test['string.5'].value_counts()

PartOfAHorizontalPortScan    685184
-                            490967
C&C                           14958
DDoS                          14395
Attack                         5962
C&C-Torii                        30
C&C-FileDownload                 14
FileDownload                      3
Name: string.5, dtype: int64

In [10]:
big = big.drop('string.5(FileDownload)', axis = 1)
big = big.drop('string.5(C&C-FileDownload)', axis = 1)
big = big.drop('string.5(C&C-Torii)', axis = 1)
big = big.drop('string.5(Attack)', axis = 1)
big = big.drop('string.5(DDoS)', axis = 1)
big = big.drop('string.5(PartOfAHorizontalPortScan)', axis = 1)
#big = big.drop('string.5(C&C)', axis = 1)
big = big.drop('string.5(-)', axis = 1)

In [11]:
big

,day,hour,addr,port,addr.1,port.1,enum(icmp),enum(tcp),enum(upd),string.1(irc),...,string.2,count.2,string.3,count.3,count.4,count.5,count.6,set[string](-),set[string]((empty)),string.5(C&C)
0,298,12,6805,58687,369334,123,0.0,0.0,1.0,0.0,...,10,0,14,1,76,1,76,1.0,0.0,0.0
1,298,12,6805,1900,420753,1900,0.0,0.0,1.0,0.0,...,6,0,7,24,8208,0,0,1.0,0.0,0.0
2,298,12,6805,32893,369335,123,0.0,0.0,1.0,0.0,...,10,0,14,1,76,1,76,1.0,0.0,0.0
3,298,12,6805,53395,316235,443,0.0,1.0,0.0,0.0,...,10,0,126,5,212,3,144,1.0,0.0,0.0
4,298,12,6805,52801,292458,53,0.0,0.0,1.0,0.0,...,10,0,14,1,62,1,339,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211508,10,21,6809,123,484566,123,0.0,0.0,1.0,0.0,...,6,0,7,1,76,0,0,1.0,0.0,0.0
1211509,10,20,6809,51834,509175,23,0.0,1.0,0.0,0.0,...,7,0,103,337,18203,362,19519,1.0,0.0,1.0
1211510,10,22,6809,123,611845,123,0.0,0.0,1.0,0.0,...,6,0,7,1,76,0,0,1.0,0.0,0.0
1211511,10,22,6809,123,646453,123,0.0,0.0,1.0,0.0,...,6,0,7,1,76,0,0,1.0,0.0,0.0


In [12]:
y = big['string.5(C&C)']
x = big.drop('string.5(C&C)', axis = 1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=10, test_size=0.7)

In [15]:
grid_tree = GridSearchCV(DecisionTreeClassifier(),
                    param_grid={'max_depth':range(1,15), \
                    'min_samples_leaf':range(1,10), \
                    'min_samples_split':range(2,12)}, 
                    cv = 3,
                    n_jobs=-1)

In [16]:
%%time
grid_tree.fit(X_train, y_train)

Wall time: 10min 5s


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': range(1, 15),
                         'min_samples_leaf': range(1, 10),
                         'm

In [20]:
grid_tree.best_params_

{'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10}

In [22]:
accuracy_score(y_test, grid_tree.predict(X_test))

0.9999787750866684

In [23]:
joblib.dump(grid_tree, 'grid_test.pkl')
# сохраняем обученные модели

['grid_test.pkl']